## 用 Python 从 web 页面获取数据

### 目标

- 了解 Python 下如何进行简单的数据爬取。


### 大纲

想要做数据的分析，第一步当时是准备数据，那现在网络上有这么多的数据，要怎么获取呢？人力一点点的复制粘贴肯定效率太低下了，通过 Python，我们可以快速的构建一个简单的 "爬虫"，从页面获取想要的数据。这里我们会做如下介绍：

1. 什么是网络爬虫，以及网络爬虫的原理
2. 如何用 python 代码获取下厨房的菜谱配料
3. 我能用这些数据做什么

## 什么是网络爬虫，以及网络爬虫的原理

**网络爬虫**，简单来说就是一个按照一定规则去下载网络上数据的程序。当我们在浏览器打开一个页面的时候，其实就是通过我们的电脑想远端的服务器请求了一些数据资源。这些数据资源依据自己的数据格式在浏览器被展示了出来。

如果我们进入浏览器的开发者模式，我们可以看到每当我们输入网址敲下回车之后都有一系列的网络请求发送了出去并获取了一些回应。你可以认为其实爬虫就是帮我们去按照指定的规则去自动的发送网络请求并获取这些网络回应。

![](chrome-network-view.png)

而我们所说的「网页」通常来说就是「HTML」页面（超文本标记语言），是一种结构化的数据。通常来说，当我们通过爬虫获取了一个网页后，我们不会需要其中的全部数据，而仅仅需要把我们所需要的特定的数据解析并以一种更规范的形式保存下来，比如我们会把指定的数据保存成 CSV，这个过程就需要我们对所获取的网页进行解析。

![](web-html-inspector.png)

这里，我们就尝试用 Python 去获取 [下厨房](https://www.xiachufang.com/category/40071/pop/) 各种菜品的配料表，并利用一些其他的工具对其进行一些可视化分析。

![](xiachufang-ingredients.png)


## 如何用 python 代码获取下厨房的菜谱配料

作为一个胶水语言，Python 自然是有很作可以完成「自动发送请求」以及「解析 HTML」工作的类库。我们其实只需要明白如何使用它就可以出色的完成我们的任务。

这里我们找到了一个名为 `requests_html` 的类库，它自己就可以帮我们完成「自动发送请求」以及「解析 HTML」这两项工作了。

首先先安装它。

In [1]:
!pip install requests_html

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
    100% |████████████████████████████████| 1.2MB 4.1MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 4.7MB/s ta 0:00:011
  Running setup.py bdist_wheel for pyppeteer ... done
  Stored in directory: /Users/rainmaker/Library/Caches/pip/wheels/3b/36/c8/82c03c30f63297ba3495a482805a61d0a4323311313799c063
  Running setup.py bdist_wheel for parse ... done
  Stored in directory: /Users/rainmaker/Library/Caches/pip/wheels/54/ed/54/30127c6fcfaf3a38332708000fb9671daab73fa0acc656f5e6
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /Users/rainmaker/Library/Caches/pip/wheels/66/f4/c8/761e9df59e7208ddc3664bb7dc7de065b7be9a39505f578935
  Running setup.py bdist_wheel for fake-useragent ... done
  Stored in directory: /Users/rainmaker/Library/Caches/pip/wheels/6d/e7/5f/3e3d35c04873ed43df30c6a3908823947ffab14fafe2d947c4
Successfully built pyppeteer parse bs4 fake-useragent
You are using pip version 18.0, h

In [16]:
from requests_html import HTMLSession

In [21]:
category_id = 40071
target_file = "早餐.csv"
max_page = 50

In [25]:
sess = HTMLSession()

results = set({})

for page in range(max_page):
    # 确定我们要抓取的网页的 URL 格式，其中 category_id 是我们预先确定好的
    # page 是每次循环都进行更新，我们这里最多获取 <max_page> 个 html 页面
    resp = sess.get(f'https://www.xiachufang.com/category/{category_id}/pop/?page={page}')
    
    if resp.status_code == 404:
        break

    # 通过 requests_html 提供的 html.find 方法获取 html 中包含菜品的数据，
    # .main-panel .info.pure-u 是 css selector，一种 html 结构定位的方法
    dishes = resp.html.find('.main-panel .info.pure-u')
    for dish in dishes:
        # 对于每一个菜品，获取它的配料
        name = dish.find('p.name > a', first=True).text
        ingredients = ','.join([ing.text for ing in dish.find('p.ing > a, p.ing > span')])
        result = (name, ingredients)
        print(result)
        results.add(result)

# 最后把所有获取的配料保存到一个 csv 文件中
with open(target_file, 'w') as f:
    f.write('\n'.join([result[1] for result in results]))
    
print(f"save file {target_file}")

('馒头成形记（图文完全版）', '面粉,酵母,白糖,水,面粉')
('葱花鸡蛋饼', '鸡蛋,面粉,葱花,盐,水')
('少女心的香蕉蔓越莓玛芬', '香蕉,鸡蛋,低粉,泡打粉,玉米油,牛奶,白砂糖,蔓越莓干')
('火腿芝士蛋三文治', '黄瓜,鸡蛋,火腿午餐肉,芝士片,三文治全麦方包')
('皮蛋瘦肉粥', '大米,皮蛋,猪瘦肉,水,盐,鸡精,料酒,淀粉,香油')
('土豆火腿早餐饼', '小香芹,土豆,香肠,胡萝卜,面粉,盐')
('热狗面包卷', '高筋面粉,火腿肠,水,细砂糖,黄油,鸡蛋液,盐,干酵母,奶粉,黑芝麻')
('【减脂早餐】无油香蕉松饼（超简单）——18.4.22更新', '香蕉（越熟越好）,鸡蛋,牛奶,面粉,盐,糖')
('菠萝包', '高筋面粉,奶粉,盐,细砂糖,鸡蛋液,酵母,水,黄油,低筋面粉,糖粉,盐,鸡蛋液,奶粉,黄油')
('淡奶油软面包', '高筋粉,低筋粉（也可以全部用高筋粉）,干酵母,细砂糖,盐,全蛋液,淡奶油,牛奶,黄油,28*28不粘烤盘,表面装饰')
('美味快手的经典美帝巧克力软曲奇', '无盐黄油,红糖,白糖,常温鸡蛋,小苏打粉,热水,盐,中筋面粉,黑巧克力豆,香草精')
('快手煎饺', '饺子（新鲜包的和速冻的皆可）,油,芝麻（选）,葱花（选）')
('葱油拌面', '食用油,葱,老抽,生抽,白糖')
('【小嶋rumi】原味松饼', '低粉,砂糖,泡打粉,牛奶,鸡蛋（室温）,黄油')
('西红柿鸡蛋疙瘩汤', '西红柿,鸡蛋,面粉,葱末,香菜末,蕃茄酱,糖,盐,香油')
('肠仔包---QQ软软的早餐包', '高筋粉,低筋粉,酵母,细砂糖,盐,蛋液,水,黄油,台湾烤肠,沙拉酱、番茄酱,香葱或者干葱碎,蛋液（刷表面用）少许')
('肉松手撕面包—超软会爆浆的面包哟', '高粉,低粉,水,牛奶,蛋液,糖,盐,酵母,黄油,奶粉')
('巧克力豆曲奇(Chocolate Chip Cookies)', '黄油,白糖,红糖,香草精,鸡蛋(或淡奶油),耐烤巧克力豆(纯可可脂70%以上的巧克力),低筋面粉,法芙娜可可粉,小苏打,盐,装饰用耐烤巧克力豆')
('绣球馒头', '面粉,酵母,牛奶')
('全麦蔓越莓奶酥包', '高筋面粉,全麦面粉,牛奶,全蛋液,淡奶油,糖,盐,酵母,黄油,黄油,糖粉（舒可曼）,全蛋液,盐,奶粉

('蒜香餐包', '高筋面粉,低筋面粉,细砂糖,盐,全蛋,无盐黄油,酵母,牛奶,水')
('胡萝卜鸡蛋饼', '面粉,鸡蛋,胡萝卜,牛奶')
('快手麻薯包', '麻薯预拌粉,鸡蛋,牛奶,黄油,砂糖,盐')
('凯撒软妹子', '面包片,奶酪,培根,圆生菜,蒜,沙拉酱,糖')
('基础早餐可丽饼', '鸡蛋,牛奶,面粉（低筋或者普通面粉均可）')
('快手鸡蛋饼～～十分钟早餐！', '鸡蛋,面粉,水,葱花,油，盐，花椒粉')
('可爱的小猪豆沙包', '普通面粉,干酵母,水,红曲粉,黑芝麻,红豆沙')
('日式土豆可乐饼', '土豆,综合冷冻蔬菜（胡萝卜,淀粉,鸡蛋,面包屑,盐,黑胡椒,沙拉酱')
('正宗蛋烘糕【中式松饼】', '清水（温水）,鸡蛋,面粉,红糖（一定要）,白砂糖,酵母粉,小苏打,盐,1、沙拉酱+肉松,2、花生酱（+肉松）,3、花生碎+芝麻碎+白砂糖,4、草莓酱,5、酸豇豆+炒肉末,6、老干妈+土豆丝,7、打发甜奶油+果粒')
('红枣糕', '红枣,面粉,鸡蛋,牛奶,核桃')
('椰蓉吐司', '高筋面粉,黄油,鸡蛋,牛奶,白糖,盐,酵母,黄油,椰蓉,白糖,鸡蛋黄,吉士粉')
('纯手工奶香馒头【无泡打粉，无添加】', '面粉,酵母粉,白糖,牛奶或清水')
('早餐鸡蛋杯cupcake Frittata', '青椒或红椒,洋葱,蘑菇,奶酪（羊奶酪，切达或你家里有的各种奶酪）,菠菜,培根,鸡蛋,橄榄油,盐和黑胡椒,帕玛森芝士磨粉')
('直接发酵法的白胖包子', '中筋面粉,水,干酵母粉,泡打粉（可用可不用）')
('葱火腿司康', '低粉,无盐黄油,牛奶,水,泡打粉,糖,盐,鸡蛋,火腿肠,小葱')
('免揉迷你吐司（藤田千秋）', '高筋面粉,清水,糖,盐,色拉油,酵母粉,高筋面粉')
('你心中的完美煎蛋', '鸡蛋')
('焗烤蔓越梅吐司布丁', '蔓越梅干,吐司,淡奶油,鸡蛋,细砂糖,香草精,黄油')
('八宝粥', '山药豆,黑米,黑豆,花生仁,小米,薏仁,百合,核桃仁,桂圆,菱角仁')
('糖油粑粑', '水磨糯米粉,水,油,红糖,白糖,蜂蜜,开水')
('简单早饭西红柿鸡蛋挂面', '鸡蛋,西红柿,挂面')
('西班牙土豆煎蛋饼', '鸡蛋,土豆,洋葱,黑胡椒粉,盐')
('胡萝卜鸡蛋饼', '小米辣椒,胡萝卜,鸡蛋,面粉,蒜,盐')


('煎饼果子', '蒜蓉酱,面粉,玉米面,鸡蛋,香菜,香葱,黑芝麻,甜面酱,油条')
('早餐白吐司 （庞多米 Pain de mie）', '三能450克吐司金模两个量,高筋面粉,细砂糖,盐,奶粉,水,即发干酵母,无盐黄油')
('早安，早餐', '面包,蔬菜,咖啡')
('献给母亲节的椰蓉蔓越莓花环面包', '高筋面粉,牛奶,淡奶油,全蛋液,酵母,糖,盐,黄油,黄油,全蛋液,糖,椰蓉,蔓越莓干')
('【我的365个早餐系列】不断更新', '一颗爱心,一颗耐心,一颗细心')
('糖不甩', '糯米粉,清水,红糖,蜂蜜,熟花生碎末')
('豆沙蛋黄面包～好吃到哭～', '高筋面粉,细砂糖,盐,酵母,鸡蛋液,牛奶,黄油,红豆沙,新鲜咸鸭蛋黄')
('爱心早餐', '耐心,爱心')
('牛奶年糕', '牛奶,年糕,蜂蜜')
('奶香玉米耙', '玉米粉,糯米粉,细砂糖,温牛奶')
('戚风蛋糕', '蛋白,柠檬汁,细砂糖,蛋黄,低筋面粉,市售橙汁,沙拉油,细砂糖')
('快手馒头伪披萨', '馒头,土鸡蛋,火腿肠,青豆或者芦笋,黑胡椒粉,孜然粉,五香粉,盐,食用油,黑芝麻粒')
('番茄鸡蛋杯（早餐鸡蛋杯升级版，彩椒鸡蛋杯姊妹篇）', '番茄,鸡蛋,青豆玉米,芝士,黑胡椒')
('Eggs Benedict 班尼迪克蛋', '英式玛芬（English Muffin）,荷兰酱（Hollandaise Sauce）,菠菜叶,培根,鸡蛋,盐,白醋（可选）,黄油')
('懒人版芝士煎蛋', '鸡蛋,芝士,盐')
('家常炒米粉', '米粉,鸡蛋,洋葱,红萝卜,火腿肠,蒜头,葱,生抽,耗油,鸡精粉,花生油')
('水果盒子蛋糕（低糖低油版）', '鸡蛋,低粉,纯牛奶（或水）,玉米油,白砂糖（蛋白）,白砂糖（蛋黄）,白醋,淡奶油,白砂糖')
('奶酥软欧包', '面团材料：,高筋面粉,牛奶,全麦面粉（或高筋面粉）,全蛋液,无盐黄油,细砂糖,酵母,盐,馅料：,奶粉,无盐黄油,细砂糖,全蛋液,蔓越莓干（葡萄干）')
('姑娘的早饭', '每天早起20分')
('椒盐花卷', '普通面粉,水,酵母,精盐，花椒粉,白糖')
('肠仔包', '热狗肠,酵母,白砂糖,盐,奶粉,黄油,全蛋,奶,高筋面粉,低筋面粉')
('椰汁马蹄千层糕（材料简单，做法简单，全家适合的Q弹Q弹233）', '马

## 我能用这些数据做什么

### 共现网络

如果某两个配料出现在了同一个菜谱中，我们就认为它存在一种「共现关系」，如果我们把所获取的配料建立一个图表会是一个什么样子呢？

这里我展示一下我利用 Gephi 做的网络可视化的结果。

![](早餐.png)